In [ ]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
# import re
import matplotlib.pyplot as plt
import csv

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
#generate 3 lists, one for all source nodes, one for all neighbours nodes, 3rd list contains 1 by default indicating linked pairs

node = []
neighbours = []
edges = []
with open("/content/drive/My Drive/Colab Notebooks/train.txt", 'r') as f:
    for data in f:
        converted_data = list(map(int, data.split()))
        for i in range(1, len(converted_data)):
            node.append(converted_data[0])
            neighbours.append(converted_data[i])
            edges.append(1)
print(max(node),len(set(node)),len(neighbours),len(set(neighbours)))

4867036 19570 24004361 4867136


In [ ]:
# print(node[:3])
# print(neighbours[:3])

[540762, 540762, 540762]
[1912140, 1537559, 3091331]


In [ ]:
#create adjacency matrix on whole dataset
from scipy.sparse import csr_matrix
source_sink_matrix=csr_matrix((edges,(node,neighbours)),shape=(4867136,4867136))

In [ ]:
# generate sample_Graph  (size: half whole graph)
def get_undirected_graph(file):
    with open(file, 'r') as train:
        G=nx.Graph() # undirected graph
        for line in train:
          if(random.random()>0.5):
            neighbour_list=[int(i) for i in line.split()]
            neighbour_tuples=[(neighbour_list[0],neighbour_list[i+1]) for i in range(len(neighbour_list)-1)]
            G.add_edges_from(neighbour_tuples)
        return G
G_sampled=get_undirected_graph('/content/drive/My Drive/Colab Notebooks/train.txt')

In [ ]:
# G_sampled_matrix=nx.to_scipy_sparse_matrix(G_sampled)

In [ ]:
print(len(G_sampled.nodes))

3345417


In [ ]:
# the function used for sampling 
def generate_2_random_numbers():
  return random.choice(node) , random.choice(neighbours)

In [ ]:
# create connected/positive and unconnected/negative node pairs
# by setting seed ==1, we sampled 20,000 connected node pairs and 20,000 unconnected node pairs 


max_sample = 20000
random.seed(10)

num_positive = 0
num_negative = 0

positiveNodePairs_sample_x = []
positive_sample_y = []


negativeNodePairs_sample_x = []
negative_sample_y = []

count=0
while num_positive < max_sample: 
    node_mid, target_mid = generate_2_random_numbers()
    # try:
    #   if G_sampled_matrix[node_mid, target_mid]== 1:
    #     break
    #   else:
    #     temp = 0
    # except:
    #   temp =0
    if source_sink_matrix[node_mid, target_mid] == 1 : # and temp ==0
        positiveNodePairs_sample_x.append((node_mid, target_mid))
        positive_sample_y.append(1)
        num_positive = num_positive + 1
        if (num_positive % 100 == 0):
          print(num_positive)            

while  num_negative < max_sample :

    node_mid, target_mid = generate_2_random_numbers()
    
    if source_sink_matrix[node_mid, target_mid] == 0:
        negativeNodePairs_sample_x.append((node_mid, target_mid))
        negative_sample_y.append(0)
        num_negative = num_negative + 1
        if (num_negative % 100 == 0):
          print(num_negative)

In [ ]:
# positiveNodePairs_sample_x[:3]
# print(source_sink_matrix[4782707, 10943])
# print(G_sampled_matrix[1580988, 2847212])
# len(positiveNodePairs_sample_x)

[(2075015, 1257998), (1927211, 2842911), (4011264, 4429091)]

In [ ]:
# print(len(positiveNodePairs_sample_x),len(negativeNodePairs_sample_x))
# print(len(positive_sample_y),len(negative_sample_y))
# print(positiveNodePairs_sample_x[:3])
# print(negativeNodePairs_sample_x[:3])

22000 22000
[(2075015, 1257998), (1927211, 2842911), (4011264, 4429091)]
[(1344759, 455930), (4841614, 197752), (3509214, 3206528)]


In [ ]:
def generate_positive_features():
    features = []
    count = 0
    print("Generating positive features......")
    for sample in positiveNodePairs_sample_x: 
        if (count % 100 == 0):
            print(count)
        count += 1
        feature = []
        try:
            preds = nx.resource_allocation_index(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.jaccard_coefficient(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.adamic_adar_index(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.preferential_attachment(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            feature.append(1)  # label=1
            
        except:
            print("one error at: "+str(count))
            pass
        features.append(feature)
    print("positive features: "+str(len(features)))
    return features


def generate_negative_features():
    features = []  
    count = 0
    print("Generating negative features......")
    for sample in negativeNodePairs_sample_x:
        if (count % 100 == 0):
            print(count)
        count += 1
        feature = []
        try:
            preds = nx.resource_allocation_index(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.jaccard_coefficient(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.adamic_adar_index(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.preferential_attachment(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            feature.append(0)  # label=0
        except:
            print("one error at: "+str(count))
            pass
        features.append(feature)
        
    print("negative features: "+str(len(features)))
    return features

In [ ]:
def generate_feature_traning_data():
    positive_features = generate_positive_features()
    negative_features = generate_negative_features()
    features = positive_features + negative_features
    # random.shuffle(features)  
    #print(features)
    return features

traning_data_features = generate_feature_traning_data()
# print(traning_data_features[:3])

In [ ]:
# nx.write_adjlist(G_sampled, "Sample_Graph.adjlist")
# G_sampled = nx.read_adjlist("Sample_Graph.adjlist")

In [ ]:
# np.save('traning_data_features.npy',np.array(traning_data_features))
# np.save('/content/drive/My Drive/Colab Notebooks/positiveNodePairs_sample_x.npy' ,np.array(positiveNodePairs_sample_x))
# np.save('/content/drive/My Drive/Colab Notebooks/negativeNodePairs_sample_x.npy', np.array(negativeNodePairs_sample_x))
# positiveNodePairs_sample_x=np.load("/content/drive/My Drive/Colab Notebooks/positiveNodePairs_sample_x.npy").tolist()
# negativeNodePairs_sample_x=np.load("/content/drive/My Drive/Colab Notebooks/negativeNodePairs_sample_x.npy").tolist()
# traning_data_features=np.load("/content/drive/My Drive/Colab Notebooks/traning_data_features.npy", allow_pickle=True).tolist()

In [ ]:
print(nx.info(G_sampled))
# print(len(positiveNodePairs_sample_x),len(negativeNodePairs_sample_x))

Name: 
Type: Graph
Number of nodes: 3345417
Number of edges: 12623601
Average degree:   7.5468


In [ ]:
def public_test_dataset():
    with open('test-public.txt') as test:
        test_edges = []
        for line in test:
            edge_list = line.split()
            try:
                test_edges.append((int(edge_list[1]), int(edge_list[2])))
            except:
                pass
        return test_edges
PublicTest_dataset_NodePairs_x = public_test_dataset()

print(PublicTest_dataset_NodePairs_x[:3])

[(3563811, 3600160), (2052043, 1401960), (4517994, 1690636)]


In [ ]:
def generate_PublicTestSet_features():
    features = []
    count = 0
    print("Generating public_test_dataset features......")
    for sample in PublicTest_dataset_NodePairs_x:
        if (count % 100 == 0):
            print(count)
        count += 1
        feature = []
        try:
            preds = nx.resource_allocation_index(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.jaccard_coefficient(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.adamic_adar_index(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.preferential_attachment(G_sampled, [sample])
            for u, v, p in preds:
                feature.append(p)
                
        except:
            print("one error at: "+str(count))
            pass
        features.append(feature)

    return features

In [ ]:
PublicTestSet_features =generate_PublicTestSet_features()

In [ ]:
#Store Sampled_NodePairds_features_2csv
def Sampled_NodePairds_features_2csv():
    with open("Sampled_NodePairds_features.csv","w",newline="") as csvfile:
        writer=csv.writer(csvfile)
        writer.writerow(["RA","JC","AA","PA","Label"])
        writer.writerows(traning_data_features)

Sampled_NodePairds_features_2csv()

In [ ]:
#Store Testdata_NodePairs_feature_2csv
def Testdata_NodePairs_feature_2csv():
    with open("publictest_NodePairs_features.csv","w",newline="") as csvfile:
        writer=csv.writer(csvfile)
        writer.writerow(["RA","JC","AA","PA"])
        writer.writerows(PublicTestSet_features)

Testdata_NodePairs_feature_2csv()

In [ ]:
print(PublicTest_dataset_NodePairs_x[:3])

[(3563811, 3600160), (2052043, 1401960), (4517994, 1690636)]


From here will be LR

In [ ]:
# Importing the dataset
dataset = pd.read_csv('Sampled_NodePairds_features.csv')
FEATURE_SIZE=4

X = dataset.iloc[:,0:FEATURE_SIZE].values
y = dataset.iloc[:, FEATURE_SIZE].values

print(X)
print(y)

[[1.09932874e-02 4.65939801e-04 4.43815134e+00 4.03452700e+06]
 [1.58791073e-02 1.50480256e-01 1.47173033e+01 1.55097000e+05]
 [9.29646566e-04 1.96926238e-04 2.20734973e+00 3.62039700e+06]
 ...
 [9.84126047e-06 2.86532951e-03 8.67383427e-02 1.34640000e+04]
 [8.93205243e-04 5.29355149e-03 1.09042457e+00 3.22728000e+05]
 [2.49792672e-06 1.65289256e-03 7.75190825e-02 2.40800000e+03]]
[1. 1. 1. ... 0. 0. 0.]


In [ ]:
# Splitting the dataset into the Training set and Test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
print(x_test)

[[-0.0169681  -0.35049375 -0.03978634 -0.03856165]
 [-0.0168465  -0.44921525 -0.02268299 -0.0265193 ]
 [-0.01696211 -0.4533219  -0.03272116 -0.02623367]
 ...
 [-0.01689427 -0.01471133 -0.028513   -0.03838246]
 [-0.01697336 -0.46927818 -0.0400373  -0.03856107]
 [-0.01697152  0.76352969 -0.03822036 -0.03856064]]


In [ ]:
from sklearn import linear_model
lr=linear_model.LogisticRegression(C=1e4,penalty='l2')
lr.fit(x_train,y_train)

LogisticRegression(C=10000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
pre=lr.predict_proba(x_test)
y_pre=[p[1] for p in pre]
acc=lr.score(x_test,y_test)
print(acc)
auc=roc_auc_score(y_test,y_pre)
print(auc)

0.6810227272727273
0.808355750804498


In [ ]:
testdata = pd.read_csv('publictest_NodePairs_features_14Sep.csv')
print(len(testdata))
x_testing = testdata.iloc[:,0:FEATURE_SIZE].values
x_testing = sc.transform(x_testing)
predictions=lr.predict_proba(x_testing)

2000


In [ ]:
with open("kaggle_submit_14Sep.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Predicted"])
    test_id=1
    for prediction in predictions:
        writer.writerow([test_id,prediction[1]])
        test_id+=1

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb

# Importing the dataset
dataset = pd.read_csv('Sampled_NodePairds_features.csv')

FEATURE_SIZE=4
X = dataset.iloc[:,0:FEATURE_SIZE].values
y = dataset.iloc[:, FEATURE_SIZE].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_val=sc.transform(x_val)

# Training
d_train = lgb.Dataset(x_train, label=y_train)
d_val = lgb.Dataset(x_val, label=y_val)
d_test=lgb.Dataset(x_test,label=y_test)

params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
#             'boosting_type': 'rf',
            'nthread': 4,
            'learning_rate': 0.02,  # 02,
            'num_leaves': 15,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.7,
            'bagging_freq': 1,
            'early_stopping_round':5,
            'max_depth':2,
            'reg_alpha': 0.041545473,
            'reg_lambda': 0.0735294,
            'min_split_gain': 0.0222415,
            'min_child_weight': 60,
            'seed': 0,
            'verbose': -1,
            'metric': 'auc',
}
clf = lgb.train(params, d_train, 20,d_val)

#Prediction
y_pred=clf.predict(x_test)
#convert into binary values
for i in range(len(y_pred)):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:
       y_pred[i]=0
    
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)
print(accuracy)

# AUC
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred))

In [ ]:
testdata = pd.read_csv('testing.csv')
print(testdata)
# x_testing = testdata.iloc[:,0:FEATURE_SIZE].values
# x_testing = sc.transform(x_testing)
# y_pred=clf.predict(x_testing)

In [ ]:
import csv
with open("kaggle_14_lightgbm.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Prediction"])
    test_id=1
    for prediction in y_pred:
        writer.writerow([test_id,prediction])
        test_id+=1